In [1]:
!pip install torch torchvision torchaudio --upgrade --quiet
!pip install diffusers==0.30.0 transformers accelerate imageio[ffmpeg] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 33.8 MB/s eta 0:00:00


In [2]:
import torch
from diffusers import CogVideoXPipeline
from diffusers.utils import export_to_video
import os
import imageio

In [3]:
import torch

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print("✅ GPU is available:", device_name)
else:
    print("❌ No GPU found. Go to Runtime > Change runtime type > select GPU")

✅ GPU is available: Tesla T4


In [4]:
prompt = """
A panda, dressed in a small, red jacket and a tiny hat, sits on a wooden stool in a
serene bamboo forest. The panda's fluffy paws strum a miniature acoustic guitar,
producing soft, melodic tunes. Nearby, a few other pandas gather, watching curiously
and some clapping in rhythm. Sunlight filters through the tall bamboo, casting a gentle
glow on the scene. The panda's face is expressive, showing concentration and joy as it plays.
The background includes a small, flowing stream and vibrant green foliage, enhancing the peaceful
and magical atmosphere of this unique musical performance.
"""

In [5]:
pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-2b",
    torch_dtype=torch.float16
)

# ✅ Safe memory optimizations (no VAE slicing/tiling here)
pipe.enable_model_cpu_offload()
pipe.enable_sequential_cpu_offload()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/783 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.39G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The config attributes {'use_rotary_positional_embeddings': False} were passed to CogVideoXTransformer3DModel, but are not expected and will be ignored. Please verify your config.json configuration file.
The config attributes {'sample_height': 480, 'sample_width': 720, 'invert_scale_latents': False} were passed to AutoencoderKLCogVideoX, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# 🎥 CogVideoX Text-to-Video Generation (12s Demo)

# Parameters
fps = 8                         # Frames per second (CogVideoX only supports fps=8)
duration_seconds = 12           # Total video duration we want (12s in this case)
segment_frames = 48             # Max frames per generation call (CogVideoX limit: ≤48 and divisible by 8)

# Each segment = segment_frames / fps seconds
# Example: 48 frames / 8 fps = 6 seconds per segment
segment_duration = segment_frames // fps

# Number of segments needed for the full video
# Example: 12s / 6s = 2 segments
num_segments = duration_seconds // segment_duration

os.makedirs("segments", exist_ok=True)

all_frames = []

# 🔁 Generate multiple segments
for i in range(num_segments):
    print(f"🎬 Generating segment {i + 1}/{num_segments}...")

    video = pipe(
        prompt=prompt,                 # The text prompt describing your scene
        num_videos_per_prompt=1,       # Generate 1 video per call
        num_inference_steps=50,        # Quality vs. speed trade-off (30 = faster, 50 = better)
        num_frames=segment_frames,     # Always <=48 and multiple of 8
        guidance_scale=6,              # Higher = closer to prompt, but less creative
        generator=torch.Generator(device="cuda").manual_seed(42 + i),  # Different seeds for variation
    ).frames[0]

    # Save each segment as an MP4
    segment_path = f"segments/segment_{i + 1}.mp4"
    export_to_video(video, segment_path, fps=fps)
    all_frames.append(segment_path)

# 📽️ Combine all segments into one continuous video
output_path = f"output_full_{duration_seconds}s.mp4"
with imageio.get_writer(output_path, fps=fps) as writer:
    for segment in all_frames:
        reader = imageio.get_reader(segment)
        for frame in reader:
            writer.append_data(frame)
        reader.close()

print(f"✅ {duration_seconds}-second video generated: {output_path}")


🎬 Generating segment 1/2...


  0%|          | 0/50 [00:00<?, ?it/s]

🎬 Generating segment 2/2...


  0%|          | 0/50 [00:00<?, ?it/s]

✅ 12-second video generated: output_full_12s.mp4


In [8]:
from google.colab import files
files.download("output_full_12s.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>